In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [4]:
hist = utils.read_df_pkl('../input/hist*0*')
new = utils.read_df_pkl('../input/new*0*')
df = pd.concat([hist, new], axis=0)

100%|██████████| 3/3 [00:26<00:00,  8.90s/it]


In [9]:
mer_cnt = df[['merchant_id', key]].drop_duplicates().groupby(['merchant_id'])[key].count()
mer_cnt.sort_values(ascending=False).head(30)

mer_cnt = mer_cnt[mer_cnt>3000]
print(mer_cnt.shape)

merchant_list = list(mer_cnt.index)
len(merchant_list)

df_m = df.loc[df.merchant_id.isin(merchant_list), ['merchant_id', 'card_id']]
print(df_m.shape)
display(df_m.head())

df_m['value'] = 1
ohe = df_m.pivot_table(index=key, columns='merchant_id', values='value', aggfunc='sum').fillna(0)
ohe.head()

(234,)

In [17]:
fname = '136_mer_'
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(ohe, how='left', on=key)
df_test = base_test.merge(ohe, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list:continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

100%|██████████| 3/3 [00:00<00:00, 35.36it/s]


In [27]:
mer_cnt = df[['merchant_category_id', key]].drop_duplicates().groupby(['merchant_category_id'])[key].count()
mer_cnt.sort_values(ascending=False).head(30)

mer_cnt = mer_cnt[mer_cnt>3000]
print(mer_cnt.shape)

merchant_list = list(mer_cnt.index)
print(len(merchant_list))

df_m = df.loc[df.merchant_category_id.isin(merchant_list), ['merchant_category_id', 'card_id']]
print(df_m.shape)
display(df_m.head())

df_m['value'] = 1
ohe = df_m.pivot_table(index=key, columns='merchant_category_id', values='value', aggfunc='sum')
ohe.fillna(0, inplace=True)
ohe.head()

merchant_category_id,2,9,14,19,33,34,40,45,57,60,63,68,69,78,80,81,87,90,105,108,110,111,114,117,126,130,157,166,171,172,178,179,180,184,195,206,210,215,216,222,225,241,248,267,273,274,278,289,290,298,299,302,307,309,315,317,319,320,330,332,333,340,342,348,356,357,360,363,367,369,373,383,385,387,391,398,400,409,411,412,414,415,417,419,420,422,427,432,434,437,443,451,454,458,462,472,480,489,491,497,506,507,511,518,519,527,528,529,531,536,544,550,554,557,560,561,574,593,605,606,607,614,623,630,631,637,642,645,650,651,661,665,667,670,671,683,690,692,695,702,703,705,706,734,737,748,755,763,769,771,774,779,783,792,793,795,796,813,818,823,829,834,836,839,840,842,843,879,884,885,891
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
C_ID_0001238066,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,1.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,0.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0
C_ID_0001506ef0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
C_ID_0001793786,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,16.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,11.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,3.0,3.0,0.0,0.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,31.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,4.0,0.0,0.0,1.0,2.0,0.0,6.0,14.0,2.0,0.0
C_ID_000183fdda,1.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,5.0,0.0,0.0,9.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.

In [28]:
ohe.columns = [col if col in ignore_list else f"merchant_category_id_{col}" for col in ohe.columns]
ohe.head()

,merchant_category_id_2,merchant_category_id_9,merchant_category_id_14,merchant_category_id_19,merchant_category_id_33,merchant_category_id_34,merchant_category_id_40,merchant_category_id_45,merchant_category_id_57,merchant_category_id_60,merchant_category_id_63,merchant_category_id_68,merchant_category_id_69,merchant_category_id_78,merchant_category_id_80,merchant_category_id_81,merchant_category_id_87,merchant_category_id_90,merchant_category_id_105,merchant_category_id_108,merchant_category_id_110,merchant_category_id_111,merchant_category_id_114,merchant_category_id_117,merchant_category_id_126,merchant_category_id_130,merchant_category_id_157,merchant_category_id_166,merchant_category_id_171,merchant_category_id_172,merchant_category_id_178,merchant_category_id_179,merchant_category_id_180,merchant_category_id_184,merchant_category_id_195,merchant_category_id_206,merchant_category_id_210,merchant_category_id_215,merchant_category_id_216,merchant_category_id_222,merchant_category_id_225,merchant_category_id_241,merchant_category_id_248,merchant_category_id_267,merchant_category_id_273,merchant_category_id_274,merchant_category_id_278,merchant_category_id_289,merchant_category_id_290,merchant_category_id_298,merchant_category_id_299,merchant_category_id_302,merchant_category_id_307,merchant_category_id_309,merchant_category_id_315,merchant_category_id_317,merchant_category_id_319,merchant_category_id_320,merchant_category_id_330,merchant_category_id_332,merchant_category_id_333,merchant_category_id_340,merchant_category_id_342,merchant_category_id_348,merchant_category_id_356,merchant_category_id_357,merchant_category_id_360,merchant_category_id_363,merchant_category_id_367,merchant_category_id_369,merchant_category_id_373,merchant_category_id_383,merchant_category_id_385,merchant_category_id_387,merchant_category_id_391,merchant_category_id_398,merchant_category_id_400,merchant_category_id_409,merchant_category_id_411,merchant_category_id_412,merchant_category_id_414,merchant_category_id_415,merchant_category_id_417,merchant_category_id_419,merchant_category_id_420,merchant_category_id_422,merchant_category_id_427,merchant_category_id_432,merchant_category_id_434,merchant_category_id_437,merchant_category_id_443,merchant_category_id_451,merchant_category_id_454,merchant_category_id_458,merchant_category_id_462,merchant_category_id_472,merchant_category_id_480,merchant_category_id_489,merchant_category_id_491,merchant_category_id_497,merchant_category_id_506,merchant_category_id_507,merchant_category_id_511,merchant_category_id_518,merchant_category_id_519,merchant_category_id_527,merchant_category_id_528,merchant_category_id_529,merchant_category_id_531,merchant_category_id_536,merchant_category_id_544,merchant_category_id_550,merchant_category_id_554,merchant_category_id_557,merchant_category_id_560,merchant_category_id_561,merchant_category_id_574,merchant_category_id_593,merchant_category_id_605,merchant_category_id_606,merchant_category_id_607,merchant_category_id_614,merchant_category_id_623,merchant_category_id_630,merchant_category_id_631,merchant_category_id_637,merchant_category_id_642,merchant_category_id_645,merchant_category_id_650,merchant_category_id_651,merchant_category_id_661,merchant_category_id_665,merchant_category_id_667,merchant_category_id_670,merchant_category_id_671,merchant_category_id_683,merchant_category_id_690,merchant_category_id_692,merchant_category_id_695,merchant_category_id_702,merchant_category_id_703,merchant_category_id_705,merchant_category_id_706,merchant_category_id_734,merchant_category_id_737,merchant_category_id_748,merchant_category_id_755,merchant_category_id_763,merchant_category_id_769,merchant_category_id_771,merchant_category_id_774,merchant_category_id_779,merchant_category_id_783,merchant_category_id_792,merchant_category_id_793,merchant_category_id_795,merchant_category_id_796,merchant_category_id_813,merchant_category_id_818,merchant_category_id_823,merchant_category_i

In [31]:
mc_cols = [col for col in ohe.columns if col.count('merchant')]
ohe['cnt'] = ohe[mc_cols].sum(axis=1)
mc_ratio_cols = [f"{col}_ratio" for col in mc_cols]
for col in mc_cols:
    ohe[f"{col}_ratio"] = ohe[col] / ohe['cnt']
ohe.head()

,merchant_category_id_2,merchant_category_id_9,merchant_category_id_14,merchant_category_id_19,merchant_category_id_33,merchant_category_id_34,merchant_category_id_40,merchant_category_id_45,merchant_category_id_57,merchant_category_id_60,merchant_category_id_63,merchant_category_id_68,merchant_category_id_69,merchant_category_id_78,merchant_category_id_80,merchant_category_id_81,merchant_category_id_87,merchant_category_id_90,merchant_category_id_105,merchant_category_id_108,merchant_category_id_110,merchant_category_id_111,merchant_category_id_114,merchant_category_id_117,merchant_category_id_126,merchant_category_id_130,merchant_category_id_157,merchant_category_id_166,merchant_category_id_171,merchant_category_id_172,merchant_category_id_178,merchant_category_id_179,merchant_category_id_180,merchant_category_id_184,merchant_category_id_195,merchant_category_id_206,merchant_category_id_210,merchant_category_id_215,merchant_category_id_216,merchant_category_id_222,merchant_category_id_225,merchant_category_id_241,merchant_category_id_248,merchant_category_id_267,merchant_category_id_273,merchant_category_id_274,merchant_category_id_278,merchant_category_id_289,merchant_category_id_290,merchant_category_id_298,merchant_category_id_299,merchant_category_id_302,merchant_category_id_307,merchant_category_id_309,merchant_category_id_315,merchant_category_id_317,merchant_category_id_319,merchant_category_id_320,merchant_category_id_330,merchant_category_id_332,merchant_category_id_333,merchant_category_id_340,merchant_category_id_342,merchant_category_id_348,merchant_category_id_356,merchant_category_id_357,merchant_category_id_360,merchant_category_id_363,merchant_category_id_367,merchant_category_id_369,merchant_category_id_373,merchant_category_id_383,merchant_category_id_385,merchant_category_id_387,merchant_category_id_391,merchant_category_id_398,merchant_category_id_400,merchant_category_id_409,merchant_category_id_411,merchant_category_id_412,merchant_category_id_414,merchant_category_id_415,merchant_category_id_417,merchant_category_id_419,merchant_category_id_420,merchant_category_id_422,merchant_category_id_427,merchant_category_id_432,merchant_category_id_434,merchant_category_id_437,merchant_category_id_443,merchant_category_id_451,merchant_category_id_454,merchant_category_id_458,merchant_category_id_462,merchant_category_id_472,merchant_category_id_480,merchant_category_id_489,merchant_category_id_491,merchant_category_id_497,...,merchant_category_id_383_ratio,merchant_category_id_385_ratio,merchant_category_id_387_ratio,merchant_category_id_391_ratio,merchant_category_id_398_ratio,merchant_category_id_400_ratio,merchant_category_id_409_ratio,merchant_category_id_411_ratio,merchant_category_id_412_ratio,merchant_category_id_414_ratio,merchant_category_id_415_ratio,merchant_category_id_417_ratio,merchant_category_id_419_ratio,merchant_category_id_420_ratio,merchant_category_id_422_ratio,merchant_category_id_427_ratio,merchant_category_id_432_ratio,merchant_category_id_434_ratio,merchant_category_id_437_ratio,merchant_category_id_443_ratio,merchant_category_id_451_ratio,merchant_category_id_454_ratio,merchant_category_id_458_ratio,merchant_category_id_462_ratio,merchant_category_id_472_ratio,merchant_category_id_480_ratio,merchant_category_id_489_ratio,merchant_category_id_491_ratio,merchant_category_id_497_ratio,merchant_category_id_506_ratio,merchant_category_id_507_ratio,merchant_category_id_511_ratio,merchant_category_id_518_ratio,merchant_category_id_519_ratio,merchant_category_id_527_ratio,merchant_category_id_528_ratio,merchant_category_id_529_ratio,merchant_category_id_531_ratio,merchant_category_id_536_ratio,merchant_category_id_544_ratio,merchant_category_id_550_ratio,merchant_category_id_554_ratio,merchant_category_id_557_ratio,merchant_category_id_560_ratio,merchant_category_id_561_ratio,merchant_category_id_574_ratio,merchant_category_id_593_ratio,merchant_category_id_605_ratio,merchant_category_id_606_ra

In [32]:
fname = '137_mer_'
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(ohe, how='left', on=key)
df_test = base_test.merge(ohe, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list:continue
    if not(col.count('ratio')) :continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

100%|██████████| 3/3 [00:00<00:00, 84.39it/s]
